# CHAPTER 9 - Quantum Support Vector Machines - Code

*Note*: You may skip the following four cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.

In [ ]:
pip install scikit-learn==1.2.1

  Using cached scikit_learn-1.2.1-cp310-cp310-win_amd64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.1-cp310-cp310-win_amd64.whl (8.3 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.7.2
    Uninstalling scikit-learn-1.7.2:
      Successfully uninstalled scikit-learn-1.7.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pennylane==0.26

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install qiskit==0.39.2

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install qiskit_machine_learning==0.5.0

In [5]:
import numpy as np

seed = 1234
np.random.seed(seed)

In [6]:
from sklearn.datasets import load_wine

x,y = load_wine(return_X_y = True)

In [7]:
x = x[:59+71]
y = y[:59+71]

In [8]:
from sklearn.model_selection import train_test_split

x_tr, x_test, y_tr, y_test = train_test_split(x, y, train_size = 0.9)

In [9]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
x_tr = scaler.fit_transform(x_tr)

In [10]:
x_test = scaler.transform(x_test)
x_test = np.clip(x_test,0,1)

In [11]:
import pennylane as qml

nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AmplitudeEmbedding(
        a, wires=range(nqubits), pad_with=0, normalize=True)
    qml.adjoint(qml.AmplitudeEmbedding(
        b, wires=range(nqubits), pad_with=0, normalize=True))
    return qml.probs(wires = range(nqubits))

c:\Users\jebar\miniconda3\envs\myenv1\lib\site-packages\pennylane\__init__.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
c:\Users\jebar\miniconda3\envs\myenv1\lib\site-packages\pennylane_lightning\lightning_qubit.py:893: UserWarning: Pre-compiled binaries for lightning.qubit are not available. Falling back to using the Python-based default.qubit implementation. To manually compile from source, follow the instructions at https://pennylane-lightning.readthedocs.io/en/latest/installation.html.
  warn(


In [12]:
from sklearn.svm import SVC
def qkernel(A, B):
    return np.array([[kernel_circ(a, b)[0] for b in B] for a in A])

svm = SVC(kernel = qkernel).fit(x_tr, y_tr)

In [13]:
from sklearn.metrics import accuracy_score

print(accuracy_score(svm.predict(x_test), y_test))

0.9230769230769231


In [14]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 8)

xs_tr = pca.fit_transform(x_tr)
xs_test = pca.transform(x_test)

In [15]:
nqubits = 8
dev = qml.device("lightning.qubit", wires=nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    qml.AngleEmbedding(a, wires=range(nqubits))
    qml.adjoint(qml.AngleEmbedding(b, wires=range(nqubits)))
    return qml.probs(wires = range(nqubits))

c:\Users\jebar\miniconda3\envs\myenv1\lib\site-packages\pennylane_lightning\lightning_qubit.py:893: UserWarning: Pre-compiled binaries for lightning.qubit are not available. Falling back to using the Python-based default.qubit implementation. To manually compile from source, follow the instructions at https://pennylane-lightning.readthedocs.io/en/latest/installation.html.
  warn(


In [16]:
svm = SVC(kernel = qkernel).fit(xs_tr, y_tr)

print(accuracy_score(svm.predict(xs_test), y_test))

1.0


In [17]:
from itertools import combinations

def ZZFeatureMap(nqubits, data):

    # Number of variables that we will load:
    # could be smaller than the number of qubits.
    nload = min(len(data), nqubits) 

    for i in range(nload):
        qml.Hadamard(i)
        qml.RZ(2.0 * data[i], wires = i)

    for pair in list(combinations(range(nload), 2)):
        q0 = pair[0]
        q1 = pair[1]

        qml.CZ(wires = [q0, q1])
        qml.RZ(2.0 * (np.pi - data[q0]) *
            (np.pi - data[q1]), wires = q1)
        qml.CZ(wires = [q0, q1])

In [18]:
nqubits = 4
dev = qml.device("lightning.qubit", wires = nqubits)

@qml.qnode(dev)
def kernel_circ(a, b):
    ZZFeatureMap(nqubits, a)
    qml.adjoint(ZZFeatureMap)(nqubits, b)
    return qml.probs(wires = range(nqubits))

svm = SVC(kernel=qkernel).fit(xs_tr, y_tr)
print(accuracy_score(svm.predict(xs_test), y_test))

c:\Users\jebar\miniconda3\envs\myenv1\lib\site-packages\pennylane_lightning\lightning_qubit.py:893: UserWarning: Pre-compiled binaries for lightning.qubit are not available. Falling back to using the Python-based default.qubit implementation. To manually compile from source, follow the instructions at https://pennylane-lightning.readthedocs.io/en/latest/installation.html.
  warn(


0.7692307692307693


In [19]:
from sklearn.metrics import accuracy_score

In [20]:
from qiskit import *

In [21]:
from qiskit.circuit import Parameter
parameter = Parameter("x")
qc = QuantumCircuit(1)
qc.rx(parameter, 0)

In [22]:
from qiskit.circuit import ParameterVector
parameter = ParameterVector("x", length = 2)
qc = QuantumCircuit(2)
qc.rx(parameter[0], 0)
qc.rx(parameter[1], 1)

In [23]:
from qiskit.circuit.library import ZZFeatureMap

zzfm = ZZFeatureMap(8)

In [24]:
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.providers.aer import AerSimulator

qkernel = QuantumKernel(feature_map = zzfm, 
            quantum_instance = AerSimulator())

In [25]:
from qiskit_machine_learning.algorithms import QSVC

qsvm = QSVC(quantum_kernel = qkernel)
qsvm.fit(xs_tr, y_tr)

QSVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
     decision_function_shape='ovr', degree=3, gamma='scale', max_iter=-1,
     probability=False,
     quantum_kernel=<qiskit_machine_learning.kernels.quantum_kernel.QuantumKernel object at 0x000002ACC3195930>,
     random_state=None, shrinking=True, tol=0.001, verbose=False)

In [26]:
print(accuracy_score(qsvm.predict(xs_test), y_test))

0.6923076923076923


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
# Using local simulator instead of IBM Quantum hardware
# No need for IBM token when using local simulators
print("Using local Aer simulator (no IBM Quantum account needed)")

In [ ]:
# Skip loading IBM account - using local simulator instead
print("Skipping IBM Quantum connection - using local simulator")

Error connecting to IBM Quantum: RequestsApiError
Please check your internet connection and try again.
Note: You can continue with local simulators without IBM Quantum access.


In [31]:
# Using local Aer simulator instead of real IBM Quantum hardware
from qiskit.providers.aer import AerSimulator

# Create a local simulator with 7 qubits
dev = AerSimulator()
print(f"Using local simulator: {dev.name()}")

Using local simulator: aer_simulator


In [32]:
from sklearn.decomposition import PCA

pca = PCA(n_components = 7)

xss_tr = pca.fit_transform(x_tr)

xss_test = pca.transform(x_test)

In [33]:
zzfm = ZZFeatureMap(7)
# Using local simulator instead of real quantum hardware
qkernel = QuantumKernel(feature_map = zzfm, quantum_instance = dev)
qsvm = QSVC(quantum_kernel = qkernel)
print("Training QSVC with local simulator...")
qsvm.fit(xss_tr, y_tr)
print("Training complete!")
print(f"Test accuracy: {accuracy_score(qsvm.predict(xss_test), y_test)}")

Training QSVC with local simulator...
Training complete!
Training complete!
Test accuracy: 0.5384615384615384
Test accuracy: 0.5384615384615384
